In [127]:
# imports and variables
import pandas as pd
import json
import altair as alt
from collections import defaultdict


file_path = 'data/result.json'
raw_data = []
user_name = "Pavlo Shulin"

In [128]:
# handle file reading
with open(file_path, 'r', encoding='utf-8') as file:
    json_data = json.load(file)


In [129]:
if 'messages' in json_data:
    messages = json_data['messages']
    raw_data = pd.DataFrame(messages)
    
    print("DataFrame created successfully!")
else:
    print("There was an error while reading data")

DataFrame created successfully!


In [130]:
reaction_user_df = raw_data[
    (raw_data['from'] == user_name) & raw_data['reactions'].notna()
]

# Display the filtered DataFrame
print(f"Total messages from {user_name} with more than 1 reaction {len(reaction_user_df)}")

Total messages from Pavlo Shulin with more than 1 reaction 115


In [131]:
# Step 1: Initialize a nested defaultdict for detailed user reactions
user_reaction_counts = defaultdict(lambda: defaultdict(int))

# Step 2: Populate the nested structure with detailed reaction data
for _, row in reaction_user_df.iterrows():
    reactions = row.get('reactions')
    if isinstance(reactions, list):
        for reaction in reactions:
            if 'recent' in reaction and isinstance(reaction['recent'], list):
                for recent_reaction in reaction['recent']:
                    user = recent_reaction.get('from')  # Extract 'from' field
                    emoji = reaction.get('emoji')  # Extract emoji
                    count = reaction.get('count', 0)  # Extract count
                    if user and emoji:  # Ensure both user and emoji exist
                        user_reaction_counts[user][emoji] += count

# Step 3: Aggregate total counts for each user
user_total_counts = {user: sum(emoji_counts.values()) for user, emoji_counts in user_reaction_counts.items()}

# Step 4: Create a DataFrame for display
reaction_summary = pd.DataFrame(list(user_total_counts.items()), columns=['User', 'Total Reaction Count'])

# Step 5: Sort the DataFrame by total count in descending order
reaction_summary = reaction_summary.sort_values(by='Total Reaction Count', ascending=False)
print("Reaction counts by user:")

display(reaction_summary)

Reaction counts by user:


,User,Total Reaction Count
3,Dmytro Bibikov,130
4,Olena Toptunenko,101
2,Pavel,77
1,Olya Baranets,67
0,Карина Коник,58
7,🏡Oleksandr Toptunenko,22
5,Sashko Kucherov,17
6,Natalie Les,13
9,Anton Bludov,9
10,Daria Panchenko,9


In [136]:
user_emoji_flat = []
for user, emojis in user_emoji_counts.items():
    anonymized_user = anonymized_user_mapping[user]
    for emoji, count in emojis.items():
        user_emoji_flat.append({'User': anonymized_user, 'Emoji': emoji, 'Count': count})

# Convert to DataFrame
user_emoji_df = pd.DataFrame(user_emoji_flat)

# Step 4: Sort and display results
user_emoji_df = user_emoji_df.sort_values(by=['User', 'Count'], ascending=[True, False])

print("Emoji Reaction Count Per User (Anonymized):")
display(user_emoji_df)

Emoji Reaction Count Per User (Anonymized):


,User,Emoji,Count
1,User 1,😁,50
0,User 1,🔥,5
2,User 1,👻,2
3,User 1,🗿,1
56,User 10,😁,8
...,...,...,...
51,User 8,💯,3
53,User 8,😱,3
50,User 8,🤝,1
54,User 9,❤,2


In [139]:
user_emoji_filtererd_df

pivot_table = user_emoji_df.pivot_table(
    index='User',      # Users as rows
    columns='Emoji',   # Emojis as columns
    values='Count',    # Reaction count as values
    aggfunc='sum',     # Sum up counts
    fill_value=0       # Replace NaN with 0 for missing emoji counts
)




In [140]:


# Example pivot_table-like data (convert your pivot_table to this format)
data = pivot_table.reset_index().melt(id_vars='User', var_name='Emoji', value_name='Reaction Count')

# Create heatmap with adjustments
heatmap = alt.Chart(data).mark_rect().encode(
    x=alt.X('Emoji:N', title="Emojis", axis=alt.Axis(labelAngle=90)),  # Rotate labels on horizontal axis
    y=alt.Y('User:N', title="Users"),   # Users as rows
    color=alt.Color('Reaction Count:Q', scale=alt.Scale(scheme='blues'), title="Reaction Count"),  # Heatmap color scale
    tooltip=['User', 'Emoji', 'Reaction Count'],  # Hover tooltips
).properties(
    title="Emoji Reaction Counts Per User",
    width=500,  # Chart width
    height=300  # Chart height
).configure_axisTop(  # Place horizontal axis legend on top
    labelFontSize=12,
    titleFontSize=14
).configure_view(
    strokeWidth=0  # Optional: Remove border around the chart
)

# Update encoding to place the horizontal axis on top
heatmap = heatmap.encode(
    x=alt.X('Emoji:N', title="Emojis", axis=alt.Axis(labelAngle=0, orient='top'))
)

heatmap.show()

alt.Chart(...)